In [1]:
import numpy as np
import os

from PIL import Image

ASSUMTION_SIGNATURE = "a"
CUSTOM_UNSAT_HEAD = ""

In [6]:
n_assumptions = 1000
n_mucs = 100

density = 0.05
verbose = 0

mucs = []

while True:
    assignment = np.random.binomial(n=1, p=density, size=n_assumptions)
    candidate = {i + 1 for i, x in enumerate(assignment) if x == 1}
    if verbose:
        print(assignment, candidate)

    if any([candidate.issubset(muc) or muc.issubset(candidate) for muc in mucs]):
        if verbose:
            print("DENIED")
    else:
        if verbose:
            print("APPROVED")
        mucs.append(candidate)
        if len(mucs) >= n_mucs:
            break

In [7]:
print(len(mucs))
for muc in mucs:
    print(muc)

assumption_dir = f"{n_assumptions}_assumptions"
muc_dir = f"{assumption_dir}/{n_mucs}_mucs"
dirname = muc_dir

if not os.path.isdir(assumption_dir):
    os.mkdir(assumption_dir)
if not os.path.isdir(muc_dir):
    os.mkdir(muc_dir)


img_width = 1280
img_height = int((n_mucs / n_assumptions) * img_width)
img = np.zeros(shape=(n_mucs, n_assumptions), dtype=np.uint8) + 255
for m, muc in enumerate(mucs):
    for a in muc:
        img[m, a - 1] = 0
Image.fromarray(img).resize((img_width, img_height), resample=Image.BOX).save(f"{dirname}/visualization.jpeg")

with open(f"{dirname}/encoding.lp", "w") as f:
    f.write("% ---[ AUTO GENERATED MUC BENCHMARK ]\n")
    f.write(f"% ---[ assumptions : {n_assumptions} ]\n")
    f.write(f"% ---[ mucs : {n_mucs} ]\n\n")

    f.write(f"num(1..{n_assumptions}).\n")
    f.write("{" + ASSUMTION_SIGNATURE + "(X): num(X)}.\n\n")

    for muc in mucs:
        f.write(f"{CUSTOM_UNSAT_HEAD}:- " + ", ".join([f"{ASSUMTION_SIGNATURE}({a})" for a in muc]) + ".\n")

with open(f"{dirname}/assumptions.lp", "w") as f:
    for a in range(n_assumptions):
        f.write(f"assume({ASSUMTION_SIGNATURE}({a + 1})).\n")

with open(f"{dirname}/results.txt", "w") as f:
    results = dict()
    results["minimal"] = [list([f"{ASSUMTION_SIGNATURE}({a})" for a in muc]) for muc in mucs]

    minimum = [muc for muc in mucs if len(muc) == len(min(mucs, key=lambda x: len(x)))]
    results["minimum"] = [list([f"{ASSUMTION_SIGNATURE}({a})" for a in muc]) for muc in minimum]

    f.write(str(results).replace("'", '"'))

with open(f"{assumption_dir}/README.md", "w") as f:
    f.write(f"# Random MUC Benchmarks [{n_assumptions} Assumptions]\n\n")
    for s, subdir in enumerate(os.walk(assumption_dir)):
        if subdir[0] != assumption_dir:
            subdir_mucs = int(subdir[0].split("/")[-1].replace("_mucs", ""))
            f.write(f"## {s} : {subdir_mucs} MUCs\n\n")
            f.write(f"![]({subdir[0].split('/')[-1]}/visualization.jpeg)\n\n")

100
{512, 129, 386, 387, 513, 6, 518, 781, 147, 663, 26, 925, 673, 422, 936, 682, 555, 557, 815, 50, 946, 315, 574, 192, 576, 70, 583, 969, 842, 204, 718, 335, 216, 217, 474, 728, 476, 732, 479, 98, 995, 615, 616, 491, 492, 878, 117, 502, 247, 380, 505, 123, 252}
{641, 130, 387, 516, 644, 137, 268, 524, 271, 403, 915, 22, 663, 798, 161, 545, 420, 293, 932, 680, 554, 174, 687, 304, 177, 947, 950, 314, 59, 572, 833, 837, 326, 583, 72, 200, 720, 465, 722, 211, 848, 597, 344, 89, 985, 347, 225, 227, 1000, 233, 875, 888, 887, 248}
{513, 898, 518, 519, 903, 777, 522, 905, 910, 655, 657, 786, 659, 660, 913, 534, 535, 543, 165, 808, 171, 814, 687, 688, 439, 313, 957, 574, 963, 836, 970, 591, 847, 82, 595, 725, 984, 986, 733, 992, 354, 358, 617, 878, 624, 497, 242, 627, 629, 246, 761, 121, 638}
{385, 130, 132, 772, 773, 647, 649, 139, 780, 655, 787, 404, 408, 666, 794, 806, 46, 302, 818, 437, 183, 823, 442, 828, 445, 703, 320, 577, 322, 833, 455, 73, 465, 721, 214, 343, 991, 226, 486, 875, 236,